In [ ]:
from datetime import datetime
import pandas as pd
import re
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import matplotlib.axes as Axes
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
import json
from pandas.io.json import json_normalize
import matplotlib as matplotlib
import warnings
warnings.filterwarnings('ignore')
from scipy import stats

In [16]:
data1415 = pd.read_csv('C:/Users/Shubham/Desktop/DSF/Project/notices14-15.tsv', sep='\t')
data1516 = pd.read_csv('C:/Users/Shubham/Desktop/DSF/Project/notices15-16.tsv', sep='\t')
data1617 = pd.read_csv('C:/Users/Shubham/Desktop/DSF/Project/notices16-17.csv')
data1718 = pd.read_csv('C:/Users/Shubham/Desktop/DSF/Project/notices17-18.csv')

In [65]:
data1415.columns

Index(['Unnamed: 0', 'state', 'id', 'link', 'county', 'newspaper', 'date',
       'content', 'notice_id'],
      dtype='object')

In [66]:
data1516.columns

Index(['Unnamed: 0', 'state', 'id', 'link', 'county', 'newspaper', 'date',
       'content', 'notice_id'],
      dtype='object')

In [67]:
data1617.columns

Index(['state', 'id', 'link', 'county', 'newspaper', 'date', 'content',
       'notice_id'],
      dtype='object')

In [68]:
data1718.columns

Index(['state', 'id', 'link', 'county', 'newspaper', 'date', 'content',
       'notice_id'],
      dtype='object')

In [69]:
data1415.drop(['Unnamed: 0'], axis=1, inplace=True)
data1516.drop(['Unnamed: 0'], axis=1, inplace=True)
data1617.drop(['Unnamed: 0'], axis=1, inplace=True)
data1718.drop(['Unnamed: 0'], axis=1, inplace=True)

In [70]:
concat = pd.concat([data1415, data1516, data1617, data1718])

In [113]:
data1718.head()

,state,id,link,county,newspaper,date,content,notice_id
0,Wyoming,3492/12012017_24466836,http://www.publicnoticeads.com/WY/search/view....,Carbon,Rawlins Daily Times,2017/12/01,PUBLIC NOTICE The Wyoming Public Service Commi...,24466836.0
1,Mississippi,691/12012017_24467322,http://www.publicnoticeads.com/MS/search/view....,Jackson,Mississippi Press,2017/12/01,"PROCLAMATION I, MOORE BEY, whose address is 34...",24467322.0
2,Arizona,72/11292017_24463039,http://www.publicnoticeads.com/AZ/search/view....,Yavapai,Verde Independent/Camp Verde Bugle,2017/12/01,DCS'S NOTICE OF HEARING ON MOTION FOR TERMINAT...,24463039.0
3,Arkansas,3141/12012017_24467983,http://www.publicnoticeads.com/AR/search/view....,Cross,Wynne Progress,2017/12/01,"IN THE CIRCUIT COURT OF CROSS COUNTY, ARKANSAS...",24467983.0
4,New Mexico,3057/12012017_24467350,http://www.publicnoticeads.com/NM/search/view....,Dona Ana,Las Cruces Sun-News,2017/12/01,"Bids can be downloaded from our website, www.g...",24467350.0


In [72]:
concat.to_csv('C:/Users/Shubham/Desktop/DSF/Project/notices14-18.csv')

In [73]:
data = concat.dropna()

data['category'] = ''

data['new_category'] = ''

data['content_lower'] = data['content'].apply(lambda x: x.lower())

In [74]:
def assign_category(content):

	content_set = set(re.split('\W', content))

	look_for_tokens = {
		'1-Bids': set(['bid', 'bids', 'bidder']),
		'2-Auctions': set(['sale', 'sales', 'sell', 'sold', 'selling', 'auction', 'auctions']),
		'3-Foreclosure': set(['foreclose', 'foreclosed', 'foreclosing', 'foreclosure', 'foreclosures', \
			'forfeit', 'forfeiting', 'forfeits', 'forefeiture', 'forefeitures', 'sized', 'seizes', 'seizure']),
		'4-Probate': set(['probate', 'death', 'deceased']),
		'5-Heir': set(['heir', 'heirs']),
		# '1-Summon_2-Appear': set(['appear', 'appearance']),
		'Creditors': set(['creditors']),
		# 'Estate': set(['estate', 'estates']),
		'Divorce': set(['divorce', 'divorced']),
		# 'Claim': set(['claim', 'claims', 'claimer', 'claimers']),
		'Government': set(['government']),
		# 'Tax': set(['tax', 'taxes']),
		# 'Debt': set(['debt', 'debts', 'indebt', 'indebted']),
		'Owed': set(['owe', 'owed']),
		'Adoption': set(['adoption']),
		'Condemnations': set(['condemn', 'condemnation']),
		'Delinquent': set(['delinquent']),
		# 'Plaintiff': set(['plaintiff', 'plaintiffs']),
		# 'Defendent': set(['defendant', 'defendants', 'defendent', 'defendents']),
		'Marriage': set(['Marriage']),
		'1-Summon': set(['summon', 'summons']),
		# 'Lien': set(['lien', 'liens']),
		'Construction': set(['construction', 'constructions']),
		'Maintenance': set(['maintenance']),
		'School': set(['school']),
		'Board': set(['board']),
		'Council': set(['council']),
		# 'Contract': set(['contract', 'contracts']),
		'Contractor': set(['contractor', 'contractors']),
		'Proposal': set(['proposal', 'proposals']),
		'2-Dependency': set(['dependency']),
		'Parenting': set(['parenting', 'parent', 'parents', 'children', 'child', 'guardianship', 'mother', 'father', 'parental']),
		'Minor': set(['minor', 'minors']),
		# 'Petition': set(['petition', 'petitions']),
		'1-Election': set(['election', 'elections']),
		'Meeting': set(['meeting', 'meetings']),
		'Citation': set(['citation', 'citations']),
		'License': set(['license', 'licenses']),
		'Vehicle': set(['vehicle', 'vehicles', 'automotive', 'automotives', 'vehicle(s)', 'autombobile', 'automobiles']),
		'Furniture': set(['furniture']),
		'Abandoned': set(['abandoned']),
		'Liquor': set(['liquor', 'alchohol', 'liquors', 'liquor', 'liquors', 'alchoholic', 'beverage', 'beverages']),
		'Storage': set(['storage']),
		'RFP/Q': set(['rfp', '(rfp)', 'rfq', '(rfq)']),
		'Purchase': set(['purchase', 'purchasing', 'purchases']),
		# 'Default': set(['default'])
	}

	look_for_phrases = {
	  	'1-AOO': set(['articles of organization']),
	  	'1-AOI': set(['articles of incorporation']),
	  	'1-AOD': set(['articles of dissolution']),
	  	'1-AOA': set(['articles of amendment']),
	  	'1-Change of Name': set(['change of name']),
	  	'Meeting2': set(['will meet on']),
	  	'Public Hearing': set(['public hearing']),
	  	'Interested Persons': set(['interested person', 'interested persons']),
	  	'Delinquent Taxes': set(['delinquent tax', 'delinquent taxes']),
	  	'Public? Storage Unit/Space': set(['storage unit', 'storage space', 'storage units', 'storage spaces', 'public storage']),
	  	# 'Public Meeting': set(['public meeting']),
	  	'Substitute Trustee': set(['substitute trustee']),
	  	'3-Default': set(['default has occurred', 'you are in default']),
	  	'3-Trust Property': set(['trust property']),
	  	'3-Child Safety': set(['child safety', 'child protection', 'children safety', 'children protection'])

	}

	categories = []

	for cat in look_for_tokens:
		finds = content_set.intersection(look_for_tokens[cat])
		if len(finds)>0:
			categories.append(cat)
		
	for cat in look_for_phrases: 
		for phrase in look_for_phrases[cat]:
			if phrase in content:
				categories.append(cat)

	return categories

start_time = datetime.now()

data['new_category'] = data['content_lower'].apply(assign_category)

end_time = datetime.now()

data['count_tags'] = data['new_category'].apply(lambda x: len(x))

data['combinations'] = data['new_category'].apply(lambda x: '_'.join(sorted(x)))

def clean_up_combinations(combination):
	if (('Bids' in combination) or ('Auctions' in combination)) and ('Default' in combination):
		return '1-Bids_2-Auctions_3-Property_4-Foreclosure_5-Default'
	# elif '1-Bids_2-Auctions_3-Foreclosure' in combination:
	# 	return '1-Bids_2-Auctions_3-Property-4-Foreclosure'
	elif (('Bids' in combination) or ('Auctions' in combination)) and ('Foreclosure' in combination):
		return '1-Bids_2-Auctions_3-Property-4-Foreclosure'
	elif (('Bids' in combination) or ('Auctions' in combination)) and (('Trust Property' in combination) or ('Substitute Trustee' in combination)):
		return '1-Bids_2-Auctions_3-Property' 
	elif (('Bids' in combination) or ('Auctions' in combination)) and ('Vehicle' in combination):
		return '1-Bids_2-Auctions_3-Vehicle'
	elif (('Bids' in combination) or ('Auctions' in combination)) and ('Furniture' in combination):
		return '1-Bids_2-Auctions_3-Furniture'
	elif ('Bids' in combination) and (('Proposal' in combination) or ('Construction' in combination) or ('Contractor' in combination)):
		return '1-Bids_2-Procuring Services'
	elif ('Bids' in combination) and ('Purchase' in combination):
		return '1-Bids_2-Seeking Purchase'
	elif (('Bids' in combination) and ('Auctions' in combination)):
		return '1-Bids_2-Auctions_3-Other'
	elif ('Bids' in combination):
		return '1-Bids_2-Auctions_3-Other'
	elif ('Auctions' in combination):
		return '1-Bids_2-Auctions_3-Other'
	elif 'Probate' in combination:
		return '1-Summon_2-Probate'
	elif ('Child Safety' in combination) and ('Dependency'):
		return '1-Summon_2-Dependency_3-Child Safety'
	elif ('Divorce' in combination) and ('Summon' in combination or 'Parenting' in combination or 'Minor' in combination):
		return '1-Summon_2-Divorce'
	elif ('Dependency' in combination) and (('Child Safety' not in combination) and ('Summon' in combination) or ('Appear' in combination)):
		return '1-Summon_2-Dependency'
	elif ('Summon' in combination) and ('Foreclosure' in combination):
		return '1-Summon_2-Foreclosure'
	elif ('Summon' in combination):
		return '1-Summon_3-Other'
	elif ('Public Hearing' in combination):
		return '1-Public Hearing'
	elif '1-AOO' in combination:
		return '1-AOO'
	elif '1-AOA' in combination:
		return '1-AOA'
	elif '1-AOD' in combination:
		return '1-AOD'
	elif '1-AOI' in combination:
		return '1-AOI'
	elif 'Meeting' in combination:
		return '1-Public Meeting'
	elif 'Change of Name' in combination:
		return '1-Change of Name'
	elif 'Parenting' in combination:
		if 'Adoption' in combination:
			return '1-Parenting_2-Adoption'
		else:
			return '1-Parenting'
	elif 'RFP/Q' in combination or 'Proposal' in combination:
		return '1-Bids_2-Procuring Services'
	elif 'Divorce' in combination:
		return '1-Summon_2-Divorce'
	elif 'Foreclosure' in combination:
		return '1-Summon_2-Foreclosure'
	else:
		return combination

data['combinations_clean'] = data['combinations'].apply(clean_up_combinations)

print(data['count_tags'].value_counts())

val_counts = data['combinations_clean'].value_counts()

3     747417
2     680902
1     671823
4     357896
0     262949
5     162986
6      91605
7      24237
8       9817
9       5189
10      3780
11      2380
12      1709
13      1311
14       884
15       560
16       457
18       375
17       319
19       209
20       104
28        98
21        94
23        56
22        39
26        17
24        15
25        11
27         5
29         3
32         1
35         1
34         1
36         1
31         1
30         1
37         1
Name: count_tags, dtype: int64


In [30]:
data.columns

Index(['state', 'id', 'link', 'county', 'newspaper', 'date', 'content',
       'notice_id', 'new_category', 'content_lower', 'category', 'count_tags',
       'combinations', 'combinations_clean'],
      dtype='object')

In [32]:
data.to_csv('C:/Users/Shubham/Desktop/DSF/Project/with_cat_1618.csv')

In [76]:
drop_con = data.drop(['content','content_lower'],axis=1)

In [156]:
drop_con = drop_con[drop_con['date']!="Printed On:"]

In [ ]:
drop_con['date'] = pd.to_datetime(drop_con['date'])
drop_con['year'] = drop_con.date.apply(lambda t: t.year)
drop_con['month'] = drop_con.date.apply(lambda t: t.month)
drop_con['day'] = drop_con.date.apply(lambda t: t.weekday())

In [166]:
len(drop_con[drop_con['combinations_clean']==''])

262860

In [161]:
drop_con.to_csv('C:/Users/Shubham/Desktop/DSF/Project/with_cat_1418.csv')

In [142]:
arizona1418 = drop_con[drop_con['state']=='Arizona']

In [143]:
arizona1418 = arizona1418[arizona1418['date']!="Printed On:"]

In [147]:
print("Size before: %d" %len(arizona1418))
arizona1418['combinations_clean'].replace('', 'Others', inplace=True)
print("Size after: %d" %len(arizona1418))

Size before: 918073
Size after: 918073


In [154]:
arizona1418.to_csv('C:/Users/Shubham/Desktop/DSF/Project/arizona1418_cleaned.csv')

In [103]:
arizona1418 = pd.read_csv('C:/Users/Shubham/Desktop/DSF/Project/arizona1418_cleaned.csv', parse_dates=["date"])

In [149]:
arizona1418['date'] = pd.to_datetime(arizona1418['date'])

In [150]:
arizona1418['year'] = arizona1418.date.apply(lambda t: t.year)
arizona1418['month'] = arizona1418.date.apply(lambda t: t.month)
arizona1418['day'] = arizona1418.date.apply(lambda t: t.weekday())

In [61]:
plot_data = arizona1618[arizona1618['combinations_clean'].isin(['1-Bids_2-Auctions_3-Property_4-Foreclosure',
                                                           '1-Bids_2-Auctions_3-Property_4-Foreclosure_5-Default',
                                                           '1-Summon_2-Foreclosure'])]

In [146]:
arizona1418 = arizona1418[arizona1418['combinations_clean']!=""]

In [145]:
len(arizona1418[arizona1418['combinations_clean']==""])

23933

In [153]:
arizona1418.dtypes

state                         object
id                            object
link                          object
county                        object
newspaper                     object
date                  datetime64[ns]
notice_id                     object
category                      object
new_category                  object
count_tags                     int64
combinations                  object
combinations_clean            object
year                           int64
month                          int64
day                            int64
dtype: object